In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import string
import json
import seaborn as sns
from scipy.stats import ttest_1samp

In [2]:
config = json.load(open('../config.json', 'r'))
INPUT_DIR = config['INPUT_DIR']
TWEET_DIR = config['TWEET_DIR']

In [3]:
events = open(INPUT_DIR + 'event_names.txt', 'r').read().splitlines()
shooter_race = json.load(open(INPUT_DIR + "shooters_race.json","r"))

In [4]:
def get_df(word):
    log_odds_list = []
    vocab_size = []
    event_names = []
    race = []
    for e in events:
        # format name
        name = ' '.join(s[0].upper() + s[1:] for s in e.split('_'))
        log_odds = np.load(TWEET_DIR +e+'/'+e+'_vocab_log_odds.npy')
        vocab = open(TWEET_DIR+e+'/'+e+'_vocab.txt', 'r').read().splitlines()
        words2idx = {w: i for i, w in enumerate(vocab)}
    
        # keep words / phrases if they occur at least ten times
        vocab_log_odds = {w: log_odds[3,i] for i, w in enumerate(vocab) if log_odds[0,i]>10}
        if word not in vocab_log_odds:
            print('missing:', name)
            continue
        
        event_names.append(name)
        vocab_size.append(log_odds.shape[1])
        log_odds_list.append(vocab_log_odds[word])
        print(name)
        race.append(shooter_race[e])
    df = pd.DataFrame.from_dict({'event': event_names, 'log_odds':log_odds_list, 'vocab_size':vocab_size, 'race':race})
    df.sort_values('vocab_size', ascending=False, inplace=True)
    print(df)
    return df

In [5]:
terr = get_df('terrorist')
terr['word'] = '"terrorist"'
craz = get_df('crazi')
craz['word'] = '"crazy"'
concat = pd.concat([terr, craz], ignore_index=True)

Chattanooga
Roseburg
Colorado Springs
San Bernardino
Kalamazoo
Orlando
Dallas
Baton Rouge
Burlington
Fort Lauderdale
Fresno
missing: San Francisco
Vegas
Thornton
Sutherland Springs
Parkland
Nashville
Santa Fe
Annapolis
Pittsburgh
Thousand Oaks
                 event  log_odds  vocab_size   race
5              Orlando  0.308304       36966    POC
11               Vegas -0.264375       26604  white
14            Parkland -0.098483        9893  white
6               Dallas  0.168449        5872    POC
13  Sutherland Springs -0.150524        4531  white
19       Thousand Oaks -0.246098        3120  white
16            Santa Fe -0.205688        3016  white
18          Pittsburgh -0.195706        2023  white
3       San Bernardino  0.184183        2010    POC
15           Nashville -0.325540        1733  white
2     Colorado Springs -0.254366        1620  white
7          Baton Rouge  0.279134        1333    POC
17           Annapolis -0.232415        1248  white
0          Chattanooga  0.09

In [48]:
f = plt.figure(dpi=400, figsize=(5,1))
ax = sns.boxplot(data=concat, x="log_odds",y='word', hue="race", orient="h", palette=sns.color_palette("Set2", 2))
plt.axvline(x=0, color='grey', linewidth=1)
plt.axhline(y=0.5, color='black', linewidth=.4, linestyle='--')
ax.set_ylabel("")
ax.spines['top'].set_linewidth(0.4)
ax.spines['right'].set_linewidth(0.4)
ax.spines['bottom'].set_linewidth(0.4)
ax.spines['left'].set_linewidth(0.4)
plt.yticks(fontsize=14)
ax.set_xlabel('Log odds ratio\n(Democrat < 0 < Republican)')
ax.text(-.3,-0.75, "Shooter's race:")
l = ax.legend(bbox_to_anchor=(.35, 1), loc=3, borderaxespad=0., ncol=2, frameon=False)
#l.set_title("Shooter's race")

In [ ]:
f = plt.figure(dpi=400, figsize=(5,1))
ax = sns.boxplot(data=df, x="log_odds",y='race', orient="h", palette=sns.color_palette("YlGnBu", 2))
ax.plot([0,0], [-1, 20], linewidth=1, color='grey')
ax.set_ylabel("Shooter's race")
ax.spines['top'].set_linewidth(0.4)
ax.spines['right'].set_linewidth(0.4)
ax.spines['bottom'].set_linewidth(0.4)
ax.spines['left'].set_linewidth(0.4)
ax.set_xlabel('Log odds ratio of "'+word+'"\n(Democrat < 0 < Republican)')

In [12]:
for i, g in df.groupby('race'):
    print(i)
    print(ttest_1samp(np.array(g['log_odds']), 0))

POC
Ttest_1sampResult(statistic=-1.7660822681325092, pvalue=0.12072527692513597)
white
Ttest_1sampResult(statistic=3.6709396041707842, pvalue=0.004310419549140847)


## Find words whose use differs the most depending on the shooter's race

In [16]:
# get all words that are not missing from more than three events
no_events = len(events)
shared_vocab = {}
for e in events:
    vocab = open(TWEET_DIR+e+'/'+e+'_vocab.txt', 'r').read().splitlines()
    for w in vocab:
        if w in shared_vocab:
            shared_vocab[w] += 1
        else:
            shared_vocab[w] = 1
shared_vocab = [k for k, v in shared_vocab.items() if v > (no_events-4)]

In [18]:
len(shared_vocab)

195

In [26]:
def get_val(vals):
    return np.abs(np.mean(vals))

In [27]:
words = []
differences = []
for w in shared_vocab:
    print(w)
    odds_white = []
    odds_poc = []
    for e in events:
        # format name
        log_odds = np.load(TWEET_DIR +e+'/'+e+'_vocab_log_odds.npy')
        vocab = open(TWEET_DIR+e+'/'+e+'_vocab.txt', 'r').read().splitlines()
        words2idx = {w: i for i, w in enumerate(vocab)}
    
        # keep words / phrases if they occur at least ten times
        vocab_log_odds = {w: log_odds[3,i] for i, w in enumerate(vocab) if log_odds[0,i]>10}
    
        if w not in vocab_log_odds:
            continue
        if shooter_race[e] == 'white':
            odds_white.append(vocab_log_odds[w])
        else:
            odds_poc.append(vocab_log_odds[w])
        
    words.append(w)
    
    differences.append(get_val(odds_white) * get_val(odds_poc) * (np.sign(np.mean(odds_white)) - np.sign(np.mean(odds_poc))))
df = pd.DataFrame.from_dict({'word': words, 'difference': differences})
print(df)

kill
shooter
famili
victim
attack
gun
4
prayer
shoot
today
terror
terrorist
obama
muslim
thought
shot
gunman
offic
peopl
dead
report
pray
thought prayer
call
news
act
live
polic
white
5
arm
die
heart
2
time
american
day
happen
free
break
god
victim famili
peac
sad
1
activ
media
friend
make
stop
blame
motiv
home
suspect
affect
offici
murder
america
law
injur
love
violenc
man
lost
work
activ shooter
prayer victim
countri
state
killer
hear
fire
guy
#break
mass
confirm
week
citi
wound
show
tweet
tragedi
cnn
control
good
talk
3
watch
involv
give
famili victim
yesterday
wait
identifi
thing
secur
place
death
incid
respons
gun control
support
updat
situat
ap
hate
press
fuck
back
start
morn
target
hope
word
mental
world
end
year
stori
care
heard
send
person
close
find
coverag
great
feel
safe
latest
rt
local
life
problem
tonight
innoc
guess
detail
read
bad
fox
crazi
open
scene
multipl
9
black
minut
post
issu
left
fact
real
shot kill
shit
wrong
crime
area
hour
hey
break news
check
hurt
lot
ppl
tw

In [28]:
df.sort_values('difference', ascending=False)

,word,difference
141,crazi,0.015278
129,latest,0.013421
175,#shoot,0.011996
188,fatal,0.011238
66,prayer victim,0.010767
142,open,0.010667
163,lot,0.009998
137,detail,0.009522
73,#break,0.009487
184,open fire,0.009369


In [29]:
df.sort_values('difference', ascending=True)

,word,difference
11,terrorist,-0.097423
13,muslim,-0.028964
10,terror,-0.026577
23,call,-0.010966
136,guess,-0.008052
159,hey,-0.005957
28,white,-0.005785
146,black,-0.005465
174,mention,-0.000794
187,shooter kill,-0.000725
